# Project1 量化金融信用评估实验

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

## Read the dataset files.

In [12]:
dir_prefix = "/home/ubuntu/workspace/data_set_public/public_data/"

train_dataset = "project_LoanStats3d_securev1.csv"
sample_dataset = "project_2015Sample.csv"
sample_df = pd.read_csv(dir_prefix + sample_dataset)
train_df = pd.read_csv(dir_prefix + train_dataset, low_memory=False) # please note that this is a large file, it may take a little bit longer to read
sample_df.head()
print 'Shape of train_df:', sample_df.shape

test_dataset = "antifraud_proj1_test.csv"
test_df = pd.read_csv(dir_prefix + test_dataset, low_memory=False)
print 'Shape of test_df:', test_df.shape
print 'Which fields are missing in the test dataset?'
print set(sample_df.columns) - set(test_df.columns)

Shape of train_df: (98, 86)
Shape of test_df: (40000, 83)
Which fields are missing in the test dataset?
set(['loan_status', 'id', 'member_id'])


## 针对Sample_df的处理

In [3]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)
sample_keys = list(sample_df.keys())
train_keys = list(train_df.keys())
del sample_keys[0],train_keys[0]
sample_df = sample_df.drop(['id','member_id'], axis = 1)#删除id，member_id

In [4]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)
sample_df['term'] = sample_df['term'].replace([' 60 months',' 36 months'],[1,0])#'term' 修改为0-1变量
#sample_df['term']

In [5]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)
del sample_df["emp_title"]#'emp_title'职业删除

In [6]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)#‘emp_length’数字化
emp_length = list(np.sort(list(sample_df['emp_length'].unique())))
#print emp_length
sample_df['emp_length']=sample_df['emp_length'].replace(['1 year', '10+ years', '2 years', '3 years', '4 years',
                                 '5 years', '6 years', '7 years', '8 years', '9 years', 
                                 '< 1 year', 'n/a'],[1,10,2,3,4,5,6,7,8,9,0,0])
#sample_df['emp_length']

In [7]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)
#sample_df['home_ownership'].unique()
sample_df['home_ownership'] = sample_df['home_ownership'].replace(['MORTGAGE', 'RENT', 'OWN'],[0,0.5,1])
#sample_df['home_ownership']

In [8]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)
#sample_df['verification_status'].unique()
sample_df['verification_status'] = sample_df['verification_status'].replace(['Source Verified', 
                                                                             'Not Verified', 'Verified'],
                                                                           [0.5,0,1])
#sample_df['verification_status']

In [9]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)#因变量Y的量化，只将charge off视作坏用户，标为“1”，current和fully paid是好用户，标为“0”
sample_df['loan_status'].unique()
sample_df = sample_df[(sample_df['loan_status']=='Current') | (sample_df['loan_status']=='Charged Off') | (sample_df['loan_status']=='Fully Paid')]
sample_df['loan_status'] = sample_df['loan_status'].replace(['Current','Fully Paid','Charged Off',],[0,0,1])
#sample_df['loan_status'] 

In [10]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)
del sample_df['pymnt_plan']#数据值只有“n”，故删除
del sample_df['desc']#描述
sample_df['purpose'] = sample_df['purpose'].replace(['car','credit_card','debt_consolidation','educational',
                                                     'home_improvement','house','major_purchase','medical',
                                                     'moving','other','renewable_energy','small_business',
                                                     'vacation','wedding'],[0.6,0.2,1.0,0.2,0.4,0.4,0.6,0.6,0.4,0.2,0.6,0.8,0.2,0.4])
del sample_df['title']#和‘purpose’字段一致
del sample_df['zip_code']#借贷程序的zip code 前三位

#sample_df['purpose']

In [11]:
train_df['addr_state'].unique()

array(['DC', 'TX', 'PA', 'GA', 'FL', 'NY', 'CA', 'TN', 'KS', 'MA', 'RI',
       'OH', 'OR', 'HI', 'SC', 'MD', 'AZ', 'WI', 'VA', 'CO', 'IN', 'LA',
       'NC', 'NJ', 'MO', 'NM', 'IL', 'MI', 'SD', 'WA', 'NH', 'VT', 'AL',
       'MN', 'CT', 'DE', 'NE', 'WV', 'MT', 'NV', 'OK', 'WY', 'AR', 'KY',
       'MS', 'ME', 'UT', 'ND', 'AK'], dtype=object)

In [12]:
#地区,我按照各地的人均GDP与总GDP，按照7:3的比例进行合计评分，并归一化
sample_df = pd.read_csv(dir_prefix + sample_dataset)
sample_df['addr_state'] = sample_df['addr_state'].replace(['DC', 'TX', 'PA', 'GA', 'FL', 'NY', 'CA', 'TN', 'KS', 'MA', 
                                                           'RI','OH', 'OR', 'HI', 'SC', 'MD', 'AZ', 'WI', 'VA', 'CO', 
                                                           'IN', 'LA','NC', 'NJ', 'MO', 'NM', 'IL', 'MI', 'SD', 'WA', 
                                                           'NH', 'VT', 'AL','MN', 'CT', 'DE', 'NE', 'WV', 'MT', 'NV',
                                                           'OK', 'WY', 'AR', 'KY','MS', 'ME', 'UT', 'ND', 'AK'],
                                                          [0.249494545,0.065555092,0.039895621,0.085444756,0.593361556,
                                                              0.190277765,0.278189313,1,0.227317672,0.192702231,0.155311576,
                                                              0.131447429,0.280336943,0.132540357,0.106452086,0.069307246,
                                                              0.15253082,0.302445071,0.21218416,0.030257468,0.137946485,
                                                              0.208228197,0.110989561,0,0.048207504,0.16335986,0.231921874,
                                                              0.161796013,0.117910048,0.2777898,0.050846682,0.094623649,
                                                              0.494175602,0.191919255,0.085791932,0.160341405,0.220064923,
                                                              0.111679786,0.046895947,0.10685987,0.109896491,0.441591068,
                                                              0.097945954,0.207944043,0.06716049,0.242232775,0.141853116,
                                                              0.013158262,0.213121431])
#sample_df['addr_state']

In [ ]:
#‘dti’每月还款金额占总贷款额百分比

In [13]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)#两年内超过三十天的逾期次数
#sample_df['delinq_2yrs']

In [235]:
#最早有开始信贷的年月
#sample_df['earliest_cr_liner']

In [14]:
#上一次逾期距今的月数，nan代表没有逾期,取0，那么月份越高说明危险度越低，取月份数的倒数
sample_df = pd.read_csv(dir_prefix + sample_dataset)
#sample_df['mths_since_last_delinq'] = [1.0/i for i in sample_df['mths_since_last_delinq']]
sample_df['mths_since_last_delinq']=sample_df['mths_since_last_delinq'].fillna(0)
#sample_df['mths_since_last_delinq']

In [15]:
#上一次有记录的月份数
sample_df['mths_since_last_record'] = sample_df['mths_since_last_record'].fillna(0)
#sample_df['mths_since_last_record'] 

In [16]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)
sample_df['revol_util'] = sample_df['revol_util'].str.strip('%').astype(float)/100
#网上将百分比转化的方法：p_float = df['p_str'].str.strip("%").astype(float)/100
#sample_df['revol_util'] 

In [17]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)
#sample_df['mths_since_recent_bc_dlq'] = [1.0/i for i in sample_df['mths_since_recent_bc_dlq']]
sample_df['mths_since_recent_bc_dlq'] = sample_df['mths_since_recent_bc_dlq'].fillna(0)

In [18]:
sample_df = pd.read_csv(dir_prefix + sample_dataset)
#sample_df['mths_since_recent_revol_delinq'] = [1.0/i for i in sample_df['mths_since_recent_revol_delinq']]
sample_df['mths_since_recent_revol_delinq'] = sample_df['mths_since_recent_revol_delinq'].fillna(0)

## 针对Train_df数据进行清理

In [13]:
train_df['term'] = train_df['term'].replace([' 60 months',' 36 months'],[1,0])#'term' 修改为0-1变量

In [14]:
train_df = train_df.drop(['id','member_id','issue_d','emp_title'], axis = 1)#删除id，member_id

In [15]:
#print emp_length
train_df['emp_length']=train_df['emp_length'].replace(['1 year', '10+ years', '2 years', '3 years', '4 years',
                                 '5 years', '6 years', '7 years', '8 years', '9 years', 
                                 '< 1 year', 'n/a'],[1,10,2,3,4,5,6,7,8,9,0,0])
#sample_df['emp_length']

In [16]:
#train_df['home_ownership'].unique()
train_df['home_ownership'] = train_df['home_ownership'].replace(['MORTGAGE', 'RENT', 'OWN','ANY'],[0,0.5,1,0.5])
#sample_df['home_ownership']

In [17]:
#sample_df['verification_status'].unique()
train_df['verification_status'] = train_df['verification_status'].replace(['Source Verified', 
                                                                             'Not Verified', 'Verified'],
                                                                           [0.5,0,1])
#sample_df['verification_status']

In [18]:
len(train_df)

421095

In [19]:
#train_df['loan_status'].unique()
train_df = train_df[(train_df['loan_status']=='Current') | (train_df['loan_status']=='Charged Off') | (train_df['loan_status']=='Fully Paid')]
#sample_df = sample_df[(sample_df['loan_status']=='Current') | (sample_df['loan_status']=='Charged Off') | (sample_df['loan_status']=='Fully Paid')]
train_df['loan_status'] = train_df['loan_status'].replace(['Current','Fully Paid','Charged Off',],[0,0,1])
#sample_df['loan_status'] 
train_df.index = range(len(train_df))

In [28]:
len(train_df)

405055

In [ ]:
#train_df.index = range(len(train_df))

In [21]:
del train_df['pymnt_plan']#数据值只有“n”，故删除
del train_df['desc']#描述
train_df['purpose'] = train_df['purpose'].replace(['car','credit_card','debt_consolidation','educational',
                                                     'home_improvement','house','major_purchase','medical',
                                                     'moving','other','renewable_energy','small_business',
                                                     'vacation','wedding'],[0.6,0.2,1.0,0.2,0.4,0.4,0.6,0.6,0.4,0.2,0.6,0.8,0.2,0.4])
del train_df['title']#和‘purpose’字段一致
del train_df['zip_code']#借贷程序的zip code 前三位

#sample_df['purpose']

In [26]:
#train_df['addr_state'].unique()

In [22]:
#地区,我按照各地的人均GDP与总GDP，按照7:3的比例进行合计评分，并归一化
train_df['addr_state'] = train_df['addr_state'].replace(['DC', 'TX', 'PA', 'GA', 'FL', 'NY', 'CA', 'TN', 'KS', 'MA', 
                                                           'RI','OH', 'OR', 'HI', 'SC', 'MD', 'AZ', 'WI', 'VA', 'CO', 
                                                           'IN', 'LA','NC', 'NJ', 'MO', 'NM', 'IL', 'MI', 'SD', 'WA', 
                                                           'NH', 'VT', 'AL','MN', 'CT', 'DE', 'NE', 'WV', 'MT', 'NV',
                                                           'OK', 'WY', 'AR', 'KY','MS', 'ME', 'UT', 'ND', 'AK'],
                                                          [0.249494545,0.065555092,0.039895621,0.085444756,0.593361556,
                                                              0.190277765,0.278189313,1,0.227317672,0.192702231,0.155311576,
                                                              0.131447429,0.280336943,0.132540357,0.106452086,0.069307246,
                                                              0.15253082,0.302445071,0.21218416,0.030257468,0.137946485,
                                                              0.208228197,0.110989561,0,0.048207504,0.16335986,0.231921874,
                                                              0.161796013,0.117910048,0.2777898,0.050846682,0.094623649,
                                                              0.494175602,0.191919255,0.085791932,0.160341405,0.220064923,
                                                              0.111679786,0.046895947,0.10685987,0.109896491,0.441591068,
                                                              0.097945954,0.207944043,0.06716049,0.242232775,0.141853116,
                                                              0.013158262,0.213121431])
#sample_df['addr_state']

In [26]:
train_df['earliest_cr_line'].to_csv('earliest_cr_line.csv')

In [27]:
#最早有开始信贷的年月 转化为开始有信贷的日子
earliest_cr_line = pd.read_csv('./earliest_cr_line.csv')
earliest_cr_line = earliest_cr_line["earliest_cr_line"]
dates = [dt.datetime.strptime(i,"%Y/%m/%d") for i in earliest_cr_line]
days = [(dt.datetime.now()-i).days for i in dates]
#days
train_df['earliest_cr_line'] = days

In [19]:
#train_df['mths_since_last_delinq'].iloc[1] =0

In [29]:
#上一次逾期距今的月数，nan代表没有逾期,取0，那么月份越高说明危险度越低，取月份数的倒数
#train_df['mths_since_last_delinq'].unique()
train_df['mths_since_last_delinq']=train_df['mths_since_last_delinq'].fillna(0)
#sample_df['mths_since_last_delinq']

In [30]:
#上一次有记录的月份数
train_df['mths_since_last_record'] = train_df['mths_since_last_record'].fillna(0)
#sample_df['mths_since_last_record'] 

In [31]:
train_df['revol_util'] = train_df['revol_util'].str.strip('%').astype(float)/100
#网上将百分比转化的方法：p_float = df['p_str'].str.strip("%").astype(float)/100
#sample_df['revol_util'] 

In [32]:
train_df['mths_since_recent_bc_dlq'] = train_df['mths_since_recent_bc_dlq'].fillna(0)

In [33]:
train_df['mths_since_recent_revol_delinq'] = train_df['mths_since_recent_revol_delinq'].fillna(0)

In [37]:
del train_df['Unnamed: 0']

In [39]:
train_df_loan_status = train_df.loan_status
train_df = train_df.drop('loan_status',axis = 1)
train_df.insert(0,'loan_status',train_df_loan_status)

### 初步清洗完的数据集已导出到“clear_train_df.csv”

In [41]:
#train_df.to_csv('clear_train_df.csv')

## 根据清理后的数据，进行归一化处理

In [43]:
cl_train_df = pd.read_csv('./clear_train_df.csv')

#del cl_train_df['Unnamed: 0']

cl_train_df = cl_train_df.drop([i for i in cl_train_keys if cl_train_df[i].count()<100000 ],axis=1)

cl_train_keys = list(cl_train_df.keys())
num_non = [[i,len(cl_train_df)-cl_train_df[i].count()] for i in cl_train_keys]

cl_train_df['num_tl_120dpd_2m'].describe()

num_non = [i for i in num_non if i[1]>0]

name_non = [num_non[i][0] for i in range(len(num_non))]

for i in name_non:
    cl_train_df[i].fillna(0,inplace = True)

## Output the results
You do not need to modify the following lines. 

**Please do not modify the file name "antifraud_proj1_result.csv".**

In [12]:
d = {'Predict': np.zeros(test_df.shape[0])}
test_predict = pd.DataFrame(data=d)
test_predict.to_csv("antifraud_proj1_result.csv")